## Data preprocessing

This notebook contains:
- Code to clean the chosen datasets
- Code to extract and engineer features:
    - TF-IDF of the tokens with the highest TF-IDF values in the training dataset
    - Flesch reading ease of the text
    - Zipfient coefficient
    - Vocabulary diversity
    - Average sentence length
    - Dot frequency
    - Comma frequency
    - Question mark frequency
    - Exclamation frequency
    - Stopword frequency
    - Noun frequency, normalized
    - Determiner frequency, normalized
    - Conjunction frequency, normalized
    - Auxiliary verbs, normalized
    - Positive sentiment score
    - Negative sentiment score
    - Neutral sentiment score
    - Compound sentiment score
    


In [20]:
# Load datasets, remove duplicates and check the size of the datasets
import pandas as pd

train_and_pretest_data = pd.read_csv('data/detect_ai.csv')
valid_data = pd.read_csv('data/detect_ai_validation.csv')
test_data = pd.read_csv('data/daigt_v4.csv')

print(f"Test data size before removing rows identical to the training data: {len(test_data)}")

train_and_pretest_data = train_and_pretest_data.drop_duplicates(subset='text')
test_data = test_data[~test_data['text'].isin(train_and_pretest_data['text'])]
valid_data = valid_data[~valid_data['text'].isin(train_and_pretest_data['text'])]

print(f"Train and pretest data size: {len(train_and_pretest_data)}")
print(f"Validation data size: {len(valid_data)}")
print(f"Test data size: {len(test_data)}")

valid_data_percentage = len(valid_data) / (len(train_and_pretest_data) + len(test_data) + len(valid_data))

print(f"The valdation data is {valid_data_percentage * 100:.2f}% of the total data")



Test data size before removing rows identical to the training data: 73573
Train and pretest data size: 158294
Validation data size: 1679
Test data size: 40202
The valdation data is 0.84% of the total data


##### Comment on valid_data:
This was first a dataset that was going to be used as validation set for the nn_model. But after making the decision to not use it as validation set we kept in in this preprocessing jupyter file because of it's small size to be able to quickly check that the preprocessing is working without having to process for 3 hours.

### Change this variable to True if you want to process all datasets
Expected runtime if True: 2-3h

Expected runtime if False: 2m

In [21]:
runAll = False

In [22]:
print(f"Train and pretest data columns: {train_and_pretest_data.columns}\n")
print(f"Validation data columns: {valid_data.columns}\n")
print(f"Test data columns: {test_data.columns}\n")
print(f"Test data[model] has the following values: {test_data['model'].unique()}\n")

Train and pretest data columns: Index(['id', 'prompt_id', 'text', 'generated'], dtype='object')

Validation data columns: Index(['id', 'prompt_id', 'text', 'generated'], dtype='object')

Test data columns: Index(['text', 'label', 'prompt_name', 'source', 'RDizzl3_seven', 'model'], dtype='object')

Test data[model] has the following values: ['human' 'mistral' 'llama' 'gpt' 'claude' 'falcon' 'palm' 'cohere' 'ada'
 'babbage' 'curie' 'davinci']



In [23]:
# Removing Irrelevant Data
# and renaming the columns to be the same for all datasets
# to ['text', 'generated'] as string, boolean for all datasets
train_and_pretest_data = train_and_pretest_data[['text', 'generated']]
valid_data = valid_data[['text', 'generated']]

test_data['generated'] = (test_data['model'] != 'human').astype(int)
test_data = test_data[['text', 'generated']]

print(f"Train and pretest data columns: {train_and_pretest_data.columns}")
print(f"Validation data columns: {valid_data.columns}")
print(f"Test data columns: {test_data.columns}")

train_and_pretest_data.head()

Train and pretest data columns: Index(['text', 'generated'], dtype='object')
Validation data columns: Index(['text', 'generated'], dtype='object')
Test data columns: Index(['text', 'generated'], dtype='object')


,text,generated
0,"In recent years, there has been a growing move...",1
1,---\nWhy not cars in our life\n===============...,1
2,A car is considered by many a nessecity for ev...,1
3,"H\n\nello fellow citezens , we are here to inf...",0
4,Have you ever known how if feels not being abl...,1


In [24]:
# Split train and pretest data into train and pretest using stratify,
# keeping the same distribution of generated values in the new datasets
from sklearn.model_selection import train_test_split

train_data, pretest_data = train_test_split(train_and_pretest_data, test_size=0.2, stratify=train_and_pretest_data['generated'])

In [25]:
# download these if it's your first time running the code
"""
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')
"""

"\nimport nltk\nnltk.download('punkt')\nnltk.download('wordnet')\nnltk.download('stopwords')\nnltk.download('averaged_perceptron_tagger')\nnltk.download('vader_lexicon')\n"

In [26]:
# Tokenization
from nltk import word_tokenize

valid_data['tokenized_text'] = valid_data['text'].apply(word_tokenize)
if runAll:
    train_data['tokenized_text'] = train_data['text'].apply(word_tokenize)
    pretest_data['tokenized_text'] = pretest_data['text'].apply(word_tokenize)
    test_data['tokenized_text'] = test_data['text'].apply(word_tokenize)

In [27]:
# Lowercasing

# Use of Co-Pilot:
# Here and in cells below, GitHub Co-pilot has been used to repeat the "runAll" pattern
# and use the same function as our code for the other 3 datasets

valid_data['tokenized_text'] = valid_data['tokenized_text'].apply(lambda x: [word.lower() for word in x])
if runAll:
    train_data['tokenized_text'] = train_data['tokenized_text'].apply(lambda x: [word.lower() for word in x])
    pretest_data['tokenized_text'] = pretest_data['tokenized_text'].apply(lambda x: [word.lower() for word in x])
    test_data['tokenized_text'] = test_data['tokenized_text'].apply(lambda x: [word.lower() for word in x])

In [28]:
# Part-of-Speech Tagging
from nltk import pos_tag
from collections import Counter

valid_data['pos_tags'] = valid_data['tokenized_text'].apply(pos_tag)

if runAll:
    train_data['pos_tags'] = train_data['tokenized_text'].apply(pos_tag)
    pretest_data['pos_tags'] = pretest_data['tokenized_text'].apply(pos_tag)
    test_data['pos_tags'] = test_data['tokenized_text'].apply(pos_tag)

# This is a function that counts specific POS tags and normalize them by the text length
def add_normalized_pos_features(df):
    def normalize_pos_counts(text_tokens):
        total_words = len(text_tokens)
        if total_words == 0:
            return {'noun_freq': 0, 'determiner_freq': 0, 'conjunction_freq': 0, 'auxiliary_freq': 0}
        
        tags = pos_tag(text_tokens)
        tag_counts = Counter(tag for word, tag in tags)
        noun_freq = sum(tag_counts[tag] for tag in ['NN', 'NNS', 'NNP', 'NNPS']) / total_words # ('NN' for singular common nouns, 'NNS' for plural common nouns, 'NNP' for singular proper nouns, 'NNPS' for plural proper nouns)
        determiner_freq = tag_counts['DT'] / total_words
        conjunction_freq = sum(tag_counts[tag] for tag in ['CC', 'IN']) / total_words #  ('CC' for coordinating conjunctions, 'IN' for subordinating or prepositions)
        auxiliary_freq = sum(tag_counts[tag] for tag in ['MD', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']) / total_words # ('MD' for modals, 'VB' for base form, 'VBD' for past tense, 'VBG' for gerund or present participle, 'VBN' for past participle, 'VBP' for non-3rd person singular present, 'VBZ' for 3rd person singular present)

        return {
            'noun_freq': noun_freq,
            'determiner_freq': determiner_freq,
            'conjunction_freq': conjunction_freq,
            'auxiliary_freq': auxiliary_freq
        }

    pos_features = df['tokenized_text'].apply(normalize_pos_counts)
    pos_features_df = pos_features.apply(pd.Series)
    for column in pos_features_df.columns:
        df[column] = pos_features_df[column]

add_normalized_pos_features(valid_data)
if runAll:
    add_normalized_pos_features(train_data)
    add_normalized_pos_features(pretest_data)
    add_normalized_pos_features(test_data)

valid_data.head()

,text,generated,tokenized_text,pos_tags,noun_freq,determiner_freq,conjunction_freq,auxiliary_freq
0,"_,_ _and it has to do with the fact that if yo...",1,"[_, ,, _, _and, it, has, to, do, with, the, fa...","[(_, NN), (,, ,), (_, FW), (_and, NN), (it, PR...",0.194175,0.082524,0.150485,0.228155
1,There are advantages of limiting car usage les...,1,"[there, are, advantages, of, limiting, car, us...","[(there, EX), (are, VBP), (advantages, NNS), (...",0.196000,0.056000,0.128000,0.228000
2,Limiting car usage ii very beneifial to not on...,1,"[limiting, car, usage, ii, very, beneifial, to...","[(limiting, VBG), (car, NN), (usage, NN), (ii,...",0.210884,0.057823,0.125850,0.200680
3,Cars have been one of the most advanced invent...,1,"[cars, have, been, one, of, the, most, advance...","[(cars, NNS), (have, VBP), (been, VBN), (one, ...",0.169082,0.072464,0.140097,0.217391
4,Are cars even really necessary? Vehicles can c...,1,"[are, cars, even, really, necessary, ?, vehicl...","[(are, VBP), (cars, NNS), (even, RB), (really,...",0.397866,0.099085,0.099085,0.102134


## Feature engineering

In [29]:
# Text Complexity and Diversity
from nltk import sent_tokenize

# Average sentence length
def average_sentence_length(text):
    sentences = sent_tokenize(text)
    if sentences:  # Check if there are any sentences to avoid division by zero
        return sum(len(word_tokenize(sentence)) for sentence in sentences) / len(sentences)
    else:
        return 0  # Return 0 if text contains no sentences

valid_data['av_sentence_length'] = valid_data['text'].apply(average_sentence_length)

if runAll:
    train_data['av_sentence_length'] = train_data['text'].apply(average_sentence_length)
    pretest_data['av_sentence_length'] = pretest_data['text'].apply(average_sentence_length)
    test_data['av_sentence_length'] = test_data['text'].apply(average_sentence_length)


# Vocabulary diversity
"""The ratio of unique words to the total number of words in a text"""
from nltk import FreqDist

valid_data['vocabulary_diversity'] = valid_data['tokenized_text'].apply(lambda x: len(set(x)) / len(x))
if runAll:
    train_data['vocabulary_diversity'] = train_data['tokenized_text'].apply(lambda x: len(set(x)) / len(x))
    pretest_data['vocabulary_diversity'] = pretest_data['tokenized_text'].apply(lambda x: len(set(x)) / len(x))
    test_data['vocabulary_diversity'] = test_data['tokenized_text'].apply(lambda x: len(set(x)) / len(x))


# Readability
# This explaination is written by ChatGPT
"""The Flesch Reading Ease score is a
readability test that provides a numerical
score indicating how easy or difficult a
text is to understand. The score is calculated
based on the average length of sentences and
the average number of syllables per word in the text.
Scores typically range from 0 to 100,
with higher scores indicating easier readability."""
from textstat import flesch_reading_ease

valid_data['flesch_reading_ease'] = valid_data['text'].apply(flesch_reading_ease)
if runAll:
    train_data['flesch_reading_ease'] = train_data['text'].apply(flesch_reading_ease)
    pretest_data['flesch_reading_ease'] = pretest_data['text'].apply(flesch_reading_ease)
    test_data['flesch_reading_ease'] = test_data['text'].apply(flesch_reading_ease)


# Zipfian Coefficient
# This explaination and function is written by ChatGPT
"""The Zipfian coefficient is a measure of the
distribution of word frequencies in a text.
It is calculated by plotting the frequency of
each word in the text against its rank in the
frequency table and fitting a curve to the data.
"""
from nltk import ngrams

def zipfian_coefficient(text, n=1):
    ngrams_list = list(ngrams(text, n))
    freq_dist = FreqDist(ngrams_list)
    freq_values = list(freq_dist.values())
    if len(freq_values) >= 2:
        freq_values.sort(reverse=True)
        return freq_values[0] / freq_values[1]
    else:
        return 0

valid_data['zipfian_coefficient'] = valid_data['tokenized_text'].apply(zipfian_coefficient)
if runAll:
    train_data['zipfian_coefficient'] = train_data['tokenized_text'].apply(zipfian_coefficient)
    pretest_data['zipfian_coefficient'] = pretest_data['tokenized_text'].apply(zipfian_coefficient)
    test_data['zipfian_coefficient'] = test_data['tokenized_text'].apply(zipfian_coefficient)


# Punctuation usage
# (They will later be normalized in the normalization part)
def add_punctuation_features(df):
    df['dot_freq'] = df['text'].str.count('\\.')
    df['comma_freq'] = df['text'].str.count(',')
    df['question_freq'] = df['text'].str.count('\\?')
    df['exclamation_freq'] = df['text'].str.count('!')

add_punctuation_features(valid_data)
if runAll:
    add_punctuation_features(train_data)
    add_punctuation_features(pretest_data)
    add_punctuation_features(test_data)


# Stopword Frequency
from nltk.corpus import stopwords

def add_stopword_count_feature(df):
    stop_words = set(stopwords.words('english'))
    df['stopword_freq'] = df['tokenized_text'].apply(lambda x: sum(1 for word in x if word in stop_words))

add_stopword_count_feature(valid_data)
if runAll:
    add_stopword_count_feature(train_data)
    add_stopword_count_feature(pretest_data)
    add_stopword_count_feature(test_data)

# New features added:
valid_data[['av_sentence_length', 'vocabulary_diversity', 'flesch_reading_ease', 'zipfian_coefficient', 'dot_freq', 'comma_freq', 'question_freq', 'exclamation_freq', 'stopword_freq']].head()

,av_sentence_length,vocabulary_diversity,flesch_reading_ease,zipfian_coefficient,dot_freq,comma_freq,question_freq,exclamation_freq,stopword_freq
0,68.666667,0.577670,77.77,1.125000,8,8,2,0,100
1,41.666667,0.432000,57.27,1.300000,6,6,0,0,118
2,21.000000,0.510204,68.81,1.142857,14,10,0,0,139
3,23.000000,0.536232,83.80,1.222222,9,3,0,0,111
4,9.939394,0.027439,96.18,1.000000,64,0,1,0,262


In [30]:
# Stop word removal
from nltk.corpus import stopwords

valid_data['tokenized_text'] = valid_data['tokenized_text'].apply(lambda x: [word for word in x if word not in stopwords.words('english')])
if runAll:
    train_data['tokenized_text'] = train_data['tokenized_text'].apply(lambda x: [word for word in x if word not in stopwords.words('english')])
    pretest_data['tokenized_text'] = pretest_data['tokenized_text'].apply(lambda x: [word for word in x if word not in stopwords.words('english')])
    test_data['tokenized_text'] = test_data['tokenized_text'].apply(lambda x: [word for word in x if word not in stopwords.words('english')])

print(f"valid_data['tokenized_text'] after removing stopwords: {valid_data['tokenized_text'].head()}")

valid_data['tokenized_text'] after removing stopwords: 0    [_, ,, _, _and, fact, automobile, ,, danger, g...
1    [advantages, limiting, car, usage, less, green...
2    [limiting, car, usage, ii, beneifial, envirome...
3    [cars, one, advanced, invention, world, ;, las...
4    [cars, even, really, necessary, ?, vehicles, c...
Name: tokenized_text, dtype: object


In [31]:
# Removal of non-alphabetic characters

valid_data['tokenized_text'] = valid_data['tokenized_text'].apply(lambda x: [word for word in x if word.isalpha()])
if runAll:
    train_data['tokenized_text'] = train_data['tokenized_text'].apply(lambda x: [word for word in x if word.isalpha()])
    pretest_data['tokenized_text'] = pretest_data['tokenized_text'].apply(lambda x: [word for word in x if word.isalpha()])    
    test_data['tokenized_text'] = test_data['tokenized_text'].apply(lambda x: [word for word in x if word.isalpha()])

In [32]:
# Lemmatization
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

valid_data['tokenized_text'] = valid_data['tokenized_text'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
if runAll:
    train_data['tokenized_text'] = train_data['tokenized_text'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
    pretest_data['tokenized_text'] = pretest_data['tokenized_text'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
    test_data['tokenized_text'] = test_data['tokenized_text'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

print(f"valid_data['tokenized_text'] after lemmatization: {valid_data['tokenized_text'].head()}")

valid_data['tokenized_text'] after lemmatization: 0    [fact, automobile, danger, getting, author, co...
1    [advantage, limiting, car, usage, le, greenhou...
2    [limiting, car, usage, ii, beneifial, envirome...
3    [car, one, advanced, invention, world, lasted,...
4    [car, even, really, necessary, vehicle, cause,...
Name: tokenized_text, dtype: object


In [33]:
# Sentiment Analysis
from nltk.sentiment import SentimentIntensityAnalyzer

valid_data['sentiment_scores'] = valid_data['text'].apply(lambda x: SentimentIntensityAnalyzer().polarity_scores(x))
sentiment_columns = valid_data['sentiment_scores'].apply(pd.Series)
valid_data = pd.concat([valid_data, sentiment_columns], axis=1)
valid_data.drop('sentiment_scores', axis=1, inplace=True)

if runAll:
    train_data['sentiment_scores'] = train_data['text'].apply(lambda x: SentimentIntensityAnalyzer().polarity_scores(x))
    sentiment_columns = train_data['sentiment_scores'].apply(pd.Series)
    train_data = pd.concat([train_data, sentiment_columns], axis=1)
    train_data.drop('sentiment_scores', axis=1, inplace=True)

    pretest_data['sentiment_scores'] = pretest_data['text'].apply(lambda x: SentimentIntensityAnalyzer().polarity_scores(x))
    sentiment_columns = pretest_data['sentiment_scores'].apply(pd.Series)
    pretest_data = pd.concat([pretest_data, sentiment_columns], axis=1)
    pretest_data.drop('sentiment_scores', axis=1, inplace=True)

    test_data['sentiment_scores'] = test_data['text'].apply(lambda x: SentimentIntensityAnalyzer().polarity_scores(x))
    sentiment_columns = test_data['sentiment_scores'].apply(pd.Series)
    test_data = pd.concat([test_data, sentiment_columns], axis=1)
    test_data.drop('sentiment_scores', axis=1, inplace=True)

valid_data[['neg', 'neu', 'pos', 'compound']].head()

,neg,neu,pos,compound
0,0.069,0.878,0.053,0.1397
1,0.069,0.832,0.099,0.8277
2,0.044,0.868,0.087,0.9392
3,0.088,0.802,0.110,0.5878
4,0.311,0.689,0.000,-0.9996


In [34]:
# TF-IDF Vectorization
from sklearn.feature_extraction.text import TfidfVectorizer

# Limit to 5000 features
vectorizer = TfidfVectorizer(max_features=5000)

if not runAll:
    valid_data['text_for_tfidf'] = valid_data['tokenized_text'].apply(lambda x: ' '.join(x))
    valid_data_sparse = vectorizer.fit_transform(valid_data['text_for_tfidf'])

if runAll: # then use the same 5000 tokens for the 3 other datasets, based on train_data
    train_data['text_for_tfidf'] = train_data['tokenized_text'].apply(lambda x: ' '.join(x))
    vectorizer.fit(train_data['text_for_tfidf'])
    train_data_sparse = vectorizer.transform(train_data['text_for_tfidf'])

    pretest_data['text_for_tfidf'] = pretest_data['tokenized_text'].apply(lambda x: ' '.join(x))
    pretest_data_sparse = vectorizer.transform(pretest_data['text_for_tfidf'])

    valid_data['text_for_tfidf'] = valid_data['tokenized_text'].apply(lambda x: ' '.join(x))
    valid_data_sparse = vectorizer.transform(valid_data['text_for_tfidf'])

    test_data['text_for_tfidf'] = test_data['tokenized_text'].apply(lambda x: ' '.join(x))
    test_data_sparse = vectorizer.transform(test_data['text_for_tfidf'])

valid_data_sparse.shape
sparse_matrices_feature_names = vectorizer.get_feature_names_out()

# The first 100 feature names
sparse_matrices_feature_names[:100]


array(['ability', 'able', 'aboard', 'abolish', 'abolished', 'abolishing',
       'absence', 'absolute', 'absolutely', 'abuse', 'abused', 'academic',
       'academically', 'accelerate', 'accelerating', 'accept',
       'acceptable', 'acceptance', 'accepted', 'access', 'accessibility',
       'accessible', 'accident', 'acciedent', 'accommodate', 'accomplish',
       'accomplished', 'accomplishment', 'according', 'accordingly',
       'account', 'accountability', 'accountable', 'accuracy', 'accurate',
       'accurately', 'accusation', 'accustomed', 'ace', 'achievable',
       'achieve', 'achieved', 'achievement', 'achieving', 'acid',
       'acknowledge', 'acknowledged', 'acknowledges', 'acknowledging',
       'acquire', 'acquired', 'acquiring', 'acquisition', 'acropolis',
       'across', 'act', 'acting', 'action', 'active', 'actively',
       'activites', 'activity', 'actor', 'actual', 'actually', 'ad',
       'adam', 'adapt', 'adaptation', 'adapting', 'add', 'added',
       'addicted

In [35]:
# Dropping all text columns and splitting into x and y
# the datasets are purely numeric after this

valid_data_y = valid_data['generated']
valid_data.drop(columns=['generated', 'text', 'tokenized_text', 'pos_tags', 'text_for_tfidf'], inplace=True)

if runAll:
    train_data_y = train_data['generated']
    train_data.drop(columns=['generated', 'text', 'tokenized_text', 'pos_tags', 'text_for_tfidf'], inplace=True)

    pretest_data_y = pretest_data['generated']
    pretest_data.drop(columns=['generated', 'text', 'tokenized_text', 'pos_tags', 'text_for_tfidf'], inplace=True)

    test_data_y = test_data['generated']
    test_data.drop(columns=['generated', 'text', 'tokenized_text', 'pos_tags', 'text_for_tfidf'], inplace=True)


In [36]:
# Normalize all features to be between 0 and 1
# based on the training data (if runall)
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

if not runAll:
    valid_data = pd.DataFrame(scaler.fit_transform(valid_data), columns=valid_data.columns)

if runAll:
    train_data = pd.DataFrame(scaler.fit_transform(train_data), columns=train_data.columns)
    pretest_data = pd.DataFrame(scaler.transform(pretest_data), columns=pretest_data.columns)
    test_data = pd.DataFrame(scaler.transform(test_data), columns=test_data.columns)
    valid_data = pd.DataFrame(scaler.transform(valid_data), columns=valid_data.columns)

valid_data.head()

,noun_freq,determiner_freq,conjunction_freq,auxiliary_freq,av_sentence_length,vocabulary_diversity,flesch_reading_ease,zipfian_coefficient,dot_freq,comma_freq,question_freq,exclamation_freq,stopword_freq,neg,neu,pos,compound
0,0.390549,0.302293,0.588925,0.399887,0.119902,0.610375,0.947123,0.003019,0.028269,0.069565,0.060606,0.0,0.080479,0.109873,0.809524,0.140957,0.569878
1,0.396010,0.202539,0.500598,0.399517,0.068056,0.451015,0.909222,0.007246,0.021201,0.052174,0.000000,0.0,0.095890,0.109873,0.736508,0.263298,0.913896
2,0.440545,0.209395,0.492153,0.334447,0.028372,0.536569,0.930558,0.003451,0.049470,0.086957,0.000000,0.0,0.113870,0.070064,0.793651,0.231383,0.969648
3,0.315471,0.264457,0.548116,0.374249,0.032212,0.565043,0.958272,0.005368,0.031802,0.026087,0.000000,0.0,0.089897,0.140127,0.688889,0.292553,0.793940
4,1.000000,0.364577,0.387015,0.099733,0.007133,0.008434,0.981160,0.000000,0.226148,0.000000,0.030303,0.0,0.219178,0.495223,0.509524,0.000000,0.000200


In [37]:
# Save the sparse matrix, DataFrames and feature names
from scipy.sparse import save_npz
import pickle

save_npz('data/valid_data_x_sparse.npz', valid_data_sparse)
valid_data.to_csv('data/valid_data_x_dense.csv', index=False)
valid_data_y.to_csv('data/valid_data_y.csv', index=False)

with open('data/sparse_matrices_feature_names.pkl', 'wb') as f:
    pickle.dump(sparse_matrices_feature_names, f)

if runAll:
    save_npz('data/train_data_x_sparse.npz', train_data_sparse)
    train_data.to_csv('data/train_data_x_dense.csv', index=False)
    train_data_y.to_csv('data/train_data_y.csv', index=False)

    save_npz('data/pretest_data_x_sparse.npz', pretest_data_sparse)
    pretest_data.to_csv('data/pretest_data_x_dense.csv', index=False)
    pretest_data_y.to_csv('pretest_data_y.csv', index=False)

    save_npz('data/test_data_x_sparse.npz', test_data_sparse)
    test_data.to_csv('data/test_data_x_dense.csv', index=False)
    test_data_y.to_csv('data/test_data_y.csv', index=False)



valid_data.head()

,noun_freq,determiner_freq,conjunction_freq,auxiliary_freq,av_sentence_length,vocabulary_diversity,flesch_reading_ease,zipfian_coefficient,dot_freq,comma_freq,question_freq,exclamation_freq,stopword_freq,neg,neu,pos,compound
0,0.390549,0.302293,0.588925,0.399887,0.119902,0.610375,0.947123,0.003019,0.028269,0.069565,0.060606,0.0,0.080479,0.109873,0.809524,0.140957,0.569878
1,0.396010,0.202539,0.500598,0.399517,0.068056,0.451015,0.909222,0.007246,0.021201,0.052174,0.000000,0.0,0.095890,0.109873,0.736508,0.263298,0.913896
2,0.440545,0.209395,0.492153,0.334447,0.028372,0.536569,0.930558,0.003451,0.049470,0.086957,0.000000,0.0,0.113870,0.070064,0.793651,0.231383,0.969648
3,0.315471,0.264457,0.548116,0.374249,0.032212,0.565043,0.958272,0.005368,0.031802,0.026087,0.000000,0.0,0.089897,0.140127,0.688889,0.292553,0.793940
4,1.000000,0.364577,0.387015,0.099733,0.007133,0.008434,0.981160,0.000000,0.226148,0.000000,0.030303,0.0,0.219178,0.495223,0.509524,0.000000,0.000200


#### How to concatinate data from saved .npz and .csv files

In [40]:
# Use this cell to load datasets and make ready for ML
import pandas as pd
from scipy.sparse import load_npz
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
import pickle

valid_data_x_sparse = load_npz('data/valid_data_x_sparse.npz')
valid_data_x_dense = pd.read_csv('data/valid_data_x_dense.csv')

with open('data/sparse_matrices_feature_names.pkl', 'rb') as f:
    sparse_matrices_feature_names = pickle.load(f)

# Combine the loaded sparse matrix with the dense matrix
valid_data_x = hstack([valid_data_x_sparse, csr_matrix(valid_data_x_dense.values)]).toarray()
valid_data_y = pd.read_csv('data/valid_data_y.csv')

print(valid_data_x.shape)
print(valid_data_y.shape)
print(sparse_matrices_feature_names.shape)

(1679, 5017)
(1679, 1)
(5000,)
